## Introduction

이 노트북은 Scikit-Learn 및 MXNet을 사용하여 Content 기반의 추천 시스템을 구축하는 방법을 보여줍니다. Collaborative Filtering 기반의 추천 시스템의 예는 다른 노트북, [mxnet-gluon-recommender](.mexnet-gluon-recommender.ipynb) 를 참고하세요. 또한, 이 노트북은 SageMaker Notebook 에서 로컬 환경의 인스턴스를 활용합니다.

이 추천 시스템은 각 뉴스 기사의 내용을 바탕으로 상위 N 개의 추천 뉴스 기사를 요청합니다.

## News Article Recommendation System

데이터셋은 하위 Kaggle 사이트에서 다운로드 받을 수 있습니다.

https://www.kaggle.com/datasets/snapcrack/all-the-news?resource=download

데이터를 다운받은 후 data 폴더를 만들고 csv 파일들을 저장합니다.

## Collaborative Filtering 과 Content-based Filtering

Collaborative Filtering 은 'cold start' 문제로 어려움을 겪습니다. 즉, 시스템에 사용자 데이터가 충분히 없는 경우 시스템에서 적절한 권장 사항을 작성하는 데 어려움이 있습니다. 일반적인 해결책은 제품데이터를 “전면 적재” 하는 것입니다. 

지금 막 시작했거나 최소한의 데이터를 보유한 비즈니스의 경우 Content-based 추천 시스템은 사용자에게 추천을 제공하는 CF 의 대안입니다.

Content-based 추천 시스템은 사용자별로 고도로 맞춤화되어 있지만 콜드 스타트 문제는 존재하지 않습니다. 이 때문에 비즈니스는 데이터가 많지 않은 초기 단계부터 추천을 제공 할 수 있습니다.

## Preparation

In [17]:
import pandas as pd
import glob
import os
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
import mxnet as mx

In [4]:
file_path = "data/"
all_files = glob.glob(file_path + "*.csv")

In [5]:
all_files

['data/articles1.csv']

In [6]:
extract_features = lambda f : pd.read_csv(f, usecols = ["id", "title", "publication", "content"])

In [7]:
articles = pd.concat((extract_features(f) for f in all_files))


In [8]:
articles = articles.head(1000)

In [9]:
articles

,id,title,publication,content
0,17283,House Republicans Fret About Winning Their Hea...,New York Times,WASHINGTON — Congressional Republicans have...
1,17284,Rift Between Officers and Residents as Killing...,New York Times,"After the bullet shells get counted, the blood..."
2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,"Death may be the great equalizer, but it isn’t..."
4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,"SEOUL, South Korea — North Korea’s leader, ..."
...,...,...,...,...
995,18412,"Russians Implicated in Doping Still Compete, A...",New York Times,"Many of the world’s winter athletes, now prepa..."
996,18413,Mother Jones Is Named Magazine of the Year - T...,New York Times,Mother Jones was named magazine of the year on...
997,18414,Florida Woman Whose ‘Stand Your Ground’ Defens...,New York Times,Marissa Alexander no longer wears an ankle mon...
998,18416,First Amendment Support Climbing Among High Sc...,New York Times,Support among American high school students fo...


## TD-IDF 생성하기

TF-IDF는 Corpus 내에서 발생하는 토큰의 수를 줄이도록 설계되었습니다. TfidfVectorizer 를 사용하면 “documents” 라고도 하는 전체 뉴스 기사 세트에서 어휘가 생성됩니다.

문서를 가져온 후 Scikit-Learn에서 TFIDFVectorizer를 정의하고 모든 기사의 내용에 대해 실행합니다. 사용되는 매개 변수는 아래에 설명되어 있으며 [scikit-Learn 설명서](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)에서 자세히 확인할 수 있습니다.

* analyzer — 피처가 단어 또는 n-그램 문자로 구성되어야 하는지 여부를 지정합니다.
* ngram_range — ngram 을 생성하기 위한 경계를 정의하며 min_n <= n <= max_n 으로 읽습니다. 이 노트북의 경우 함께 발생할 수 있는 최대 단어는 3,“trigram” 입니다. 만약, 두 단어만 함께 나오는 것을 찾고 있다면 범위를 (1,2) 로 지정하면 “bigram”이 생성됩니다.
* min_df — 문서 빈도가 지정된 임계치보다 낮은 단어를 무시하기 위한 cutoff 값입니다. 이 노트북에서는 문서의 20% 미만에서 단어가 나오는 경우 해당 단어가 제거됩니다. 값이 클수록 보다 aggressive 한 텍스트 필터링이 생성됩니다.
* stop_words — 문제의 맥락에서 가치가 없는 것으로 간주되는 단어입니다. 영어에서 가장 일반적인 단어는 “I”, “me”, “the”, “and”등과 같은 단어를 포함하며, 이 노트북에서는 영어로 된 일반적인 중지 단어를 필터링합니다. 특정 문제에 대한 사용자 지정 단어 목록을 만들 수도 있습니다.

<code>fit_transform</code> 함수는 NumPy 희소 행렬 (Sparse Matrix) 을 반환합니다. 즉, 이 유형의 행렬에서 대부분의 요소는 0입니다. 요소의 대부분이 0이 아니면 dense 행렬로 간주됩니다. 

In [13]:
%%time


tf = TfidfVectorizer(analyzer="word", ngram_range=(1, 3),min_df=0.2, stop_words="english")

tfidf_matrix = tf.fit_transform(articles["content"])

CPU times: user 2.25 s, sys: 144 ms, total: 2.39 s
Wall time: 2.39 s


In [32]:
tfidf_matrix

<1000x189 sparse matrix of type '<class 'numpy.float64'>'
	with 59338 stored elements in Compressed Sparse Row format>

## Create recommnedation with MXNet

MXNet은 고도로 최적화되어 Machine Learning 과 관련된 수학적 계산에 적합합니다. 또한, MXNet은 NumPy보다 빠르게 데이터를 처리 할 수 있습니다.

잠시 NumPy와 MXNet의 속도를 비교해 보겠습니다.

In [55]:
%timeit np.dot(tfidf_matrix, tfidf_matrix.T)

68.8 ms ± 56.7 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [34]:
%%time

mx_tfidf = mx.nd.sparse.array(tfidf_matrix, ctx=mx.cpu())

CPU times: user 5.75 ms, sys: 0 ns, total: 5.75 ms
Wall time: 4.69 ms


In [52]:
%%timeit

mx.nd.sparse.dot(mx_tfidf, mx_tfidf.T)
mx.nd.waitall()

3.19 ms ± 195 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


위 결과에서 알수 있듯이 numpy 를 사용할 때 보다 mxnet 을 사용할 때 더 좋은 성능을 확인할 수 있습니다.

note: 실험 결과는 SageMaker 의 노트북 instance type 과 환경에 따라 다소 달라질 수 있습니다.

## Recommendation

cosine similarity 행렬이 생성되면 이제 사용자에게 추천을 할 수 있습니다. 이 노트북에서는 content 에서 만든 추천을 제공하지만 title 에서 권장 사항을 기반으로 합니다. 먼저 콘솔에 처음 10개의 title 을 인쇄합니다.

In [56]:
def get_recommendations(df_articles, article_idx, mx_mat, n_recs=10):
    """
    Request top N article recommendations.

    INPUT
        df_articles: Pandas DataFrame containing all articles.
        user_id: User ID being provided matches.
        mx_mat: MXNet cosine similarity matrix
    OUTPUT
        Pandas DataFrame of top N article recommendations.
    """

    # Similarity and recommendations
    article_sims = mx_mat[article_idx].asnumpy()
    article_recs = np.argsort(-article_sims).tolist()[:n_recs + 1]

    # Top recommendations
    df_recs = df_articles.iloc[article_recs]
    df_recs["similarity"] = article_sims[article_recs]

    return df_recs

In [26]:
mx_recsys = mx.nd.sparse.dot(mx_tfidf, mx_tfidf.T)

In [28]:
df_recs = get_recommendations(df_articles = articles,
    article_idx = 3, mx_mat = mx_recsys, n_recs=10)

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [30]:
df_recs

,id,title,publication,content,similarity
3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,"Death may be the great equalizer, but it isn’t...",1.000000
65,17360,My Canada - The New York Times,New York Times,"Canada, our No. 1 pick for this year’s 52 Plac...",0.595626
658,18023,"Love and Black Lives, in Pictures Found on a B...",New York Times,"One night six years ago, on a quiet side stree...",0.540853
740,18134,Postcard From My Past: Crossing Into Syria - T...,New York Times,"Seventeen years ago, the Middle East was a mor...",0.518828
941,18354,Review: ‘Lincoln in the Bardo’ Shows a Preside...,New York Times,LINCOLN IN THE BARDOBy George Saunders343 page...,0.513843
239,17553,Danielle Brooks: The First Time I Saw Myself o...,New York Times,"It was June 2012, and I had just sped out of t...",0.511749
270,17588,"Neanderthals Were People, Too - The New York T...",New York Times,Joachim Neander was a Calvinist theologian w...,0.509455
469,17812,"‘I’m Nobody’? Not a Chance, Emily Dickinson - ...",New York Times,"“In the Trumpian sense of the term, she’s the ...",0.503491
206,17518,Is Edward Snowden a Spy? A New Book Calls Him ...,New York Times,"HOW AMERICA LOST ITS SECRETSEdward Snowden, th...",0.494410
196,17506,Here’s How Clemson Won the College Football Pl...,New York Times,Clemson upset Alabama and won the college foot...,0.479991
